In [1]:
!pip install helpers

In [2]:
#IMPORTING LIBRARIES

import numpy as np
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sent = SentimentIntensityAnalyzer()

from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
import seaborn as sns
import helpers # custom module



In [3]:
#LOADING THE DATASET
df=pd.read_csv("../input/beer-data-analytics/BeerProject.csv",encoding='latin-1')

#Reading number of columns and rows in data
print(f"{df.shape[0]} rows, {df.shape[1]} columns")
df.head()

528870 rows, 13 columns


,beer_ABV,beer_beerId,beer_brewerId,beer_name,beer_style,review_appearance,review_palette,review_overall,review_taste,review_profileName,review_aroma,review_text,review_time
0,5.0,47986,10325,Sausa Weizen,Hefeweizen,2.5,2.0,1.5,1.5,stcules,1.5,A lot of foam. But a lot. In the smell some ba...,1234817823
1,6.2,48213,10325,Red Moon,English Strong Ale,3.0,2.5,3.0,3.0,stcules,3.0,"Dark red color, light beige foam, average. In ...",1235915097
2,6.5,48215,10325,Black Horse Black Beer,Foreign / Export Stout,3.0,2.5,3.0,3.0,stcules,3.0,"Almost totally black. Beige foam, quite compac...",1235916604
3,5.0,47969,10325,Sausa Pils,German Pilsener,3.5,3.0,3.0,2.5,stcules,3.0,"Golden yellow color. White, compact foam, quit...",1234725145
4,7.7,64883,1075,Cauldron DIPA,American Double / Imperial IPA,4.0,4.5,4.0,4.0,johnmichaelsen,4.5,"According to the website, the style for the Ca...",1293735206


**Data Analysis**
For performing EDA, we can either use automated tools like autoviz or can use inbuilt pandas functions to get the idea of how the data looks like.
Below I have used pandas for data exploration.

In [4]:
#Having a look at the Data
df.describe()

,beer_ABV,beer_beerId,beer_brewerId,review_appearance,review_palette,review_overall,review_taste,review_aroma,review_time
count,508590.000000,528870.000000,528870.000000,528870.000000,528870.000000,528870.000000,528870.000000,528870.000000,5.288700e+05
mean,7.017442,22098.466016,2598.423429,3.864522,3.758926,3.833197,3.765993,3.817350,1.224885e+09
std,2.204460,22158.284352,5281.805350,0.604010,0.685335,0.709962,0.669018,0.718903,7.605600e+07
min,0.010000,3.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,8.843904e+08
25%,5.300000,1745.000000,132.000000,3.500000,3.500000,3.500000,3.500000,3.500000,1.174613e+09
50%,6.500000,14368.000000,394.000000,4.000000,4.000000,4.000000,4.000000,4.000000,1.240366e+09
75%,8.500000,40528.000000,1475.000000,4.000000,4.000000,4.500000,4.000000,4.500000,1.288560e+09
max,57.700000,77310.000000,27980.000000,5.000000,5.000000,5.000000,5.000000,5.000000,1.326277e+09


In [5]:
#Checking for null values
(df.isnull().sum()/len(df))*100

beer_ABV              3.834591
beer_beerId           0.000000
beer_brewerId         0.000000
beer_name             0.000000
beer_style            0.000000
review_appearance     0.000000
review_palette        0.000000
review_overall        0.000000
review_taste          0.000000
review_profileName    0.021744
review_aroma          0.000000
review_text           0.022501
review_time           0.000000
dtype: float64

From the above data we can see that 3 columns have missing values. Out of which, beer_ABV has the highest percentage of missing values.

In [6]:
#filling the null values in review_profileName, review_text with mode
df['review_profileName'].fillna(df['review_profileName'].mode()[0], inplace=True)
df['review_text'].fillna(df['review_text'].mode()[0], inplace=True)
df.isnull().sum()

beer_ABV              20280
beer_beerId               0
beer_brewerId             0
beer_name                 0
beer_style                0
review_appearance         0
review_palette            0
review_overall            0
review_taste              0
review_profileName        0
review_aroma              0
review_text               0
review_time               0
dtype: int64

To fill beer_ABV column null values, methods like mode, median can't be adopted since for each unique beer, there exist a different beer_ABV. 
Thus to fill those values, I will take out non null dataset first, find the beer_ABV value for each beer type and then map them in the dataset

In [7]:
#Taking out non null dataset
df_notnull = df.loc[df.beer_ABV.notna(),['beer_name','beer_ABV']].sort_values(by = 'beer_name')
df_notnull.head()

,beer_name,beer_ABV
432191,"""100"" Pale Ale",6.6
327824,"""33"" Export",4.8
327823,"""33"" Export",4.8
327825,"""33"" Export",4.8
322080,"""76"" Anniversary Ale",7.6


In [8]:
#Getting beer_ABV value for each category and storing in dictionary
beer_mean_abv = df_notnull.groupby('beer_name')['beer_ABV'].mean().to_dict()


In [9]:
#Mapping values for each category
df.beer_ABV = df.beer_name.map(beer_mean_abv)


In [10]:
#Mapping values 
beer_mean_abv_fin = df.groupby('beer_name')['beer_ABV'].mean().to_dict()
#beer_mean_abv_fin

In [11]:
df.beer_ABV = df.beer_name.map(beer_mean_abv_fin)
df.isnull().sum()

beer_ABV              17920
beer_beerId               0
beer_brewerId             0
beer_name                 0
beer_style                0
review_appearance         0
review_palette            0
review_overall            0
review_taste              0
review_profileName        0
review_aroma              0
review_text               0
review_time               0
dtype: int64

In [12]:
#For beer types that don't have any beer_ABV value have been dropped.
df.dropna(inplace=True)
df.isnull().sum()

beer_ABV              0
beer_beerId           0
beer_brewerId         0
beer_name             0
beer_style            0
review_appearance     0
review_palette        0
review_overall        0
review_taste          0
review_profileName    0
review_aroma          0
review_text           0
review_time           0
dtype: int64

The beers that have a high ABV(Alcohol by volume) are said to be strong beers. To find the breweries which have strongest beer_ABV value, I have simply done a group by by "beer_brewerId" and the the mean of "beer_ABV"

In [13]:
#strong_beers_df = df.groupby('beer_brewerId')['beer_ABV'].mean()
#strong_beers_df
strong_beers_df = df.groupby('beer_brewerId')['beer_ABV'].mean()
strong_beers_df = pd.DataFrame(data=strong_beers_df).sort_values(by=['beer_ABV'],ascending=False).reset_index()
strong_beers_df.head(3)

,beer_brewerId,beer_ABV
0,6513,19.228824
1,736,13.750000
2,24215,12.466667


**The beer brewer id's 6513,736,24215 produce the strongest beers.**

In [14]:
#convert unix timestamp to python timestamp
df['timestamp']=pd.to_datetime(df['review_time'], unit="s")
df.head()

,beer_ABV,beer_beerId,beer_brewerId,beer_name,beer_style,review_appearance,review_palette,review_overall,review_taste,review_profileName,review_aroma,review_text,review_time,timestamp
0,5.0,47986,10325,Sausa Weizen,Hefeweizen,2.5,2.0,1.5,1.5,stcules,1.5,A lot of foam. But a lot. In the smell some ba...,1234817823,2009-02-16 20:57:03
1,6.2,48213,10325,Red Moon,English Strong Ale,3.0,2.5,3.0,3.0,stcules,3.0,"Dark red color, light beige foam, average. In ...",1235915097,2009-03-01 13:44:57
2,6.5,48215,10325,Black Horse Black Beer,Foreign / Export Stout,3.0,2.5,3.0,3.0,stcules,3.0,"Almost totally black. Beige foam, quite compac...",1235916604,2009-03-01 14:10:04
3,5.0,47969,10325,Sausa Pils,German Pilsener,3.5,3.0,3.0,2.5,stcules,3.0,"Golden yellow color. White, compact foam, quit...",1234725145,2009-02-15 19:12:25
4,7.7,64883,1075,Cauldron DIPA,American Double / Imperial IPA,4.0,4.5,4.0,4.0,johnmichaelsen,4.5,"According to the website, the style for the Ca...",1293735206,2010-12-30 18:53:26


In [15]:
#Extracting year from timestamp
df['year'] = pd.DatetimeIndex(df['timestamp']).year
df.head()

,beer_ABV,beer_beerId,beer_brewerId,beer_name,beer_style,review_appearance,review_palette,review_overall,review_taste,review_profileName,review_aroma,review_text,review_time,timestamp,year
0,5.0,47986,10325,Sausa Weizen,Hefeweizen,2.5,2.0,1.5,1.5,stcules,1.5,A lot of foam. But a lot. In the smell some ba...,1234817823,2009-02-16 20:57:03,2009
1,6.2,48213,10325,Red Moon,English Strong Ale,3.0,2.5,3.0,3.0,stcules,3.0,"Dark red color, light beige foam, average. In ...",1235915097,2009-03-01 13:44:57,2009
2,6.5,48215,10325,Black Horse Black Beer,Foreign / Export Stout,3.0,2.5,3.0,3.0,stcules,3.0,"Almost totally black. Beige foam, quite compac...",1235916604,2009-03-01 14:10:04,2009
3,5.0,47969,10325,Sausa Pils,German Pilsener,3.5,3.0,3.0,2.5,stcules,3.0,"Golden yellow color. White, compact foam, quit...",1234725145,2009-02-15 19:12:25,2009
4,7.7,64883,1075,Cauldron DIPA,American Double / Imperial IPA,4.0,4.5,4.0,4.0,johnmichaelsen,4.5,"According to the website, the style for the Ca...",1293735206,2010-12-30 18:53:26,2010


In [16]:
#Finding year in which beers enjoyed the highest ratings
highest_rating_year=df.sort_values(by=['review_overall'], ascending=False)
highest_rating_year['year']

33907     2007
418389    2010
521423    2011
418423    2009
236902    2008
          ... 
267500    2007
441284    2010
39638     2002
39617     2002
39601     2002
Name: year, Length: 510950, dtype: int64

In [17]:
#Finding which features out of taste,aroma, appearance and palette are important for user's ratings
df_beer_feature=df[['review_taste','review_aroma','review_appearance','review_palette','review_overall']]
df_beer_feature

,review_taste,review_aroma,review_appearance,review_palette,review_overall
0,1.5,1.5,2.5,2.0,1.5
1,3.0,3.0,3.0,2.5,3.0
2,3.0,3.0,3.0,2.5,3.0
3,2.5,3.0,3.5,3.0,3.0
4,4.0,4.5,4.0,4.5,4.0
...,...,...,...,...,...
528792,4.0,4.0,4.0,3.5,4.5
528793,3.5,3.5,4.0,3.5,3.5
528794,3.0,3.0,3.5,3.5,3.0
528795,3.0,3.0,3.0,3.0,3.5


In [18]:
#Plotting correlation matrix
df_beer_feature.corr()

,review_taste,review_aroma,review_appearance,review_palette,review_overall
review_taste,1.000000,0.723056,0.551939,0.601100,0.689629
review_aroma,0.723056,1.000000,0.531321,0.703602,0.780588
review_appearance,0.551939,0.531321,1.000000,0.544819,0.483247
review_palette,0.601100,0.703602,0.544819,1.000000,0.598400
review_overall,0.689629,0.780588,0.483247,0.598400,1.000000


**From above correlation we can see that review_aroma has the highest value and thus can say it plays an important role for customers while chosing a beer.**

To find out which beer styles are preferred based on customer's reviews, we can calculate a sentiment score for the reviews and put them in some categories on the basis that the higher the sentiment score, the more famous the beer style is among customers.

In [19]:
#Creating a dataframe reviews 
reviews=[]
reviews=pd.DataFrame(reviews)
reviews['review_text']=df['review_text']
reviews.head()

,review_text
0,A lot of foam. But a lot. In the smell some ba...
1,"Dark red color, light beige foam, average. In ..."
2,"Almost totally black. Beige foam, quite compac..."
3,"Golden yellow color. White, compact foam, quit..."
4,"According to the website, the style for the Ca..."


In [ ]:
polarity = [round(sent.polarity_scores(i)['compound'], 2) for i in reviews['review_text']]
reviews['sentiment_score'] = polarity
reviews.head()

In [20]:
'''The above cell takes some time to calculate the sentiment score for reviews. thus I have calculated
them once and saved in a excel sheet.'''

#Loading the reviews dataframe with sentiment score
reviews=pd.read_csv('../input/reviews/reviews_df.csv')
reviews.head()

,Unnamed: 0,review_text,sentiment_score,beer_style
0,0,A lot of foam. But a lot. In the smell some ba...,0.19,Hefeweizen
1,1,"Dark red color, light beige foam, average. In ...",0.59,English Strong Ale
2,2,"Almost totally black. Beige foam, quite compac...",0.30,Foreign / Export Stout
3,3,"Golden yellow color. White, compact foam, quit...",0.98,German Pilsener
4,4,"According to the website, the style for the Ca...",0.95,American Double / Imperial IPA


In [22]:
top_beer_styles=reviews[reviews.sentiment_score==1]
top_beer_styles.beer_style.unique()

array(['Rauchbier', 'American Pale Ale (APA)', 'Russian Imperial Stout',
       'American Adjunct Lager', 'American IPA',
       'American Amber / Red Ale', 'English Bitter', 'English Strong Ale',
       'American Porter', 'Tripel', 'Dunkelweizen', 'Dubbel',
       'American Black Ale', 'American Double / Imperial IPA',
       'Czech Pilsener', 'Schwarzbier', 'Hefeweizen',
       'English Barleywine', 'Winter Warmer', 'Belgian Strong Pale Ale',
       'Pumpkin Ale', 'Belgian IPA', 'Black & Tan', 'American Barleywine',
       'Belgian Strong Dark Ale', 'Belgian Pale Ale',
       'American Double / Imperial Stout', 'Scottish Ale',
       'Extra Special / Strong Bitter (ESB)', 'Bock', 'Belgian Dark Ale',
       'Old Ale', 'Bière de Garde', 'English Porter', 'English Brown Ale',
       'Herbed / Spiced Beer', 'Oatmeal Stout', 'Altbier',
       'American Stout', 'Weizenbock', 'Lambic - Unblended',
       'Scotch Ale / Wee Heavy', 'American Strong Ale',
       'American Brown Ale', 'English 

From above we can see the beer styles that are preferred by many customers.
Some other ways to calculate sentiment score can be:
1. Averaging with overall review score
2. Taking review_text as train_features and overall_review as target_feature, and apply regression to calculate an overall rating for them
3. Uisng the count of negative and positive words in the data

**To find out what could be the top 3 beers that can be recommended, I have tried to make use of sentiment_score,overall_review and beer_ABV. Adding these columns in the dataframe, and doing a group by by beer style, and finally sorting on the basis of these 3 to find top recommendations.**

In [23]:
#Adding sentiment score to dataframe
df['polarity']=reviews['sentiment_score']

In [31]:
#calculation mean beer_abv 
beers_df=df.groupby('beer_style')['beer_ABV'].mean()
beers_df = pd.DataFrame(data=beers_df)
beers_df.head()

,beer_ABV
beer_style,
Altbier,5.942592
American Adjunct Lager,4.846224
American Amber / Red Ale,6.236761
American Amber / Red Lager,4.899944
American Barleywine,10.305371


In [32]:
##calculation mean sentiment_score 
beers_polarity=df.groupby('beer_style')['polarity'].mean()
beers_polarity=pd.DataFrame(data=beers_polarity)
beers_polarity.head()

,polarity
beer_style,
Altbier,0.792467
American Adjunct Lager,0.740329
American Amber / Red Ale,0.765794
American Amber / Red Lager,0.721470
American Barleywine,0.798640


In [33]:
#calculation mean revuew_score 
beers_review=df.groupby('beer_style')['review_overall'].mean()
beers_review=pd.DataFrame(data=beers_review)
beers_review.head()

,review_overall
beer_style,
Altbier,3.877912
American Adjunct Lager,3.137958
American Amber / Red Ale,3.876303
American Amber / Red Lager,3.394155
American Barleywine,3.920452


In [34]:
#add columns for average beer_abv, sentiment polarity, review_overall
recommneded_beers=[]
recommneded_beers=pd.DataFrame(recommneded_beers)
recommneded_beers=pd.concat([beers_df,beers_review,beers_polarity])
recommneded_beers.head()

,beer_ABV,review_overall,polarity
beer_style,,,
Altbier,5.942592,NaN,NaN
American Adjunct Lager,4.846224,NaN,NaN
American Amber / Red Ale,6.236761,NaN,NaN
American Amber / Red Lager,4.899944,NaN,NaN
American Barleywine,10.305371,NaN,NaN


In [35]:
recommneded_beers['review_overall']=beers_review['review_overall']
recommneded_beers.head()

,beer_ABV,review_overall,polarity
beer_style,,,
Altbier,5.942592,3.877912,NaN
American Adjunct Lager,4.846224,3.137958,NaN
American Amber / Red Ale,6.236761,3.876303,NaN
American Amber / Red Lager,4.899944,3.394155,NaN
American Barleywine,10.305371,3.920452,NaN


In [36]:
recommneded_beers['polarity']=beers_polarity['polarity']
recommneded_beers.head()

,beer_ABV,review_overall,polarity
beer_style,,,
Altbier,5.942592,3.877912,0.792467
American Adjunct Lager,4.846224,3.137958,0.740329
American Amber / Red Ale,6.236761,3.876303,0.765794
American Amber / Red Lager,4.899944,3.394155,0.721470
American Barleywine,10.305371,3.920452,0.798640


In [37]:
recommneded_beers.sort_values(by=['beer_ABV','review_overall','polarity'], ascending=False)


,beer_ABV,review_overall,polarity
beer_style,,,
Eisbock,12.546141,4.082474,0.756495
Wheatwine,11.550227,3.816327,0.795659
American Strong Ale,11.025980,3.733773,0.801109
Quadrupel (Quad),10.941370,4.052010,0.699094
English Barleywine,10.684988,3.778606,0.779489
...,...,...,...
Euro Strong Lager,NaN,2.814227,0.768861
American Malt Liquor,NaN,2.724907,0.787167
American Malt Liquor,NaN,2.724907,0.787167


The top 3 beers that can be recommended are:
1. Eisbock
2. Wheatwine
3. American Strong Ale

In [39]:
df_compare=df[['polarity','review_overall']]
df_compare.head()

,polarity,review_overall
0,0.19,1.5
1,0.59,3.0
2,0.30,3.0
3,0.98,3.0
4,0.95,4.0


**From above Dataframe what we can observe is the higher the review is, the more is the sentiment score for that. Sentiment score is calculated for the written review and thus we can find this similarity between them.**

**How can we find similar beer drinkers using written reviews only?**
- To answer this, some of the similarity measures like cosine similarity, jaccard similarity, using bert for contextual similarity have to be used followed by clustering the same vector groups.
- If we go with BERT encodings and then find simialarity between the text reviews then it would give us the best of clusters of similar type of beer drinkers. 
- But it's not easy to perform this operation on normal machines since it would require high amount of space and computation speed. Thus I haven't been able to do this part.